In [1]:
from ydata_profiling import ProfileReport
import pandas as pd
import numpy as np
import pandas_datareader as pdr  # to install: !pip install pandas_datareader
from datetime import datetime
url = 'https://github.com/LeDataSciFi/data/raw/main/Firm%20Year%20Datasets%20(Compustat)/firms2020.csv'
firms_df = pd.read_csv(url).drop_duplicates('tic')

bod_df = pd.read_csv('./Input_data/BOD_Comp_Company_Fin.csv')
ceo_df = pd.read_csv('./Input_data/CEO_Comp_Fin.csv')
sp500_tickers = pd.read_csv('./Input_data/sp500_2022.csv')['Symbol'].tolist()
test_df = pd.read_csv('./Input_data/directorstest.csv')
df = pd.read_csv('./Input_data/ExecuComp_data.csv')

/var/folders/29/vl12lkgs28xgjpc6t0_jg9040000gn/T/ipykernel_1890/3821301156.py:13: DtypeWarning: Columns (80,96) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./Input_data/ExecuComp_data.csv')


In [5]:
# bod_df = bod_df.drop(['NAICS', 'PCOMMFEES', 'PEXECDIRPD', 'EPSEX', 'EPSEXCHG', 'EPSEX3LS', 'EPSEX5LS', 'OIBD3LS'], axis=1)
# bod_df = bod_df.drop(['OIBDCHG', 'OIBD5LS', 'SALE3LS', 'SALE5LS', 'NI3LS', 'NI5LS', 'PDIRPENSN'], axis=1)
# bod_df = bod_df.drop(['NUMMTGS', 'TRS1YR', 'TRS3YR', 'TRS5YR', 'AJEX', 'BS_YIELD', 'BS_VOLATILITY'], axis=1)

BOD_df = bod_df[['GVKEY', 'YEAR', 'CUSIP', 'TICKER', 'SIC', 'ANNDIRRET', 
       'DIRMTGFEE', 'DIROPT', 'DIROPTAD', 'DIRSTK', 'DIRSTKAD', 
       'PDIRPENSN', 'OIBD', 'PRETAX', 'NIBEX', 'MKTVAL']]

In [3]:
ceo_df = ceo_df.drop(['CFOANN', 'PENSION_CHG', 'TOTAL_SEC', 'CHG_CTRL_PYMT', 'TOTAL_SEC_PCT', 'REPRICE', 'PCFO', 'EXECRANK'], axis=1)
ceo_df = ceo_df[ceo_df['CEOANN'].notnull()]
BOTH_df = ceo_df.merge(BOD_df, on = 'TICKER', how = 'left')

In [15]:
# BOTH_df.info()
# TDC1 is total compensation for all CEOs
#ANNDIRRET is for directors (their main salary) 


In [18]:
# assuming your DataFrame is named 'df'
BOTH_df['director_comp'] = (BOTH_df['ANNDIRRET'] + BOTH_df['DIRMTGFEE'] + BOTH_df['DIROPT'] 
                                   + BOTH_df['DIROPTAD'] + BOTH_df['DIRSTK'] + BOTH_df['DIRSTKAD'] + 
                                   (BOTH_df['PDIRPENSN'] * BOTH_df['ANNDIRRET'])
                                  )

In [23]:
grouped_df = BOTH_df.groupby('TICKER')['director_comp'].sum().reset_index()

merged_df = pd.merge(BOTH_df, grouped_df, on='TICKER', how='left')

merged_df = merged_df.rename(columns={'director_comp_': 'total_director_comp'})


,EXECDIR,CO_PER_ROL,SALARY,BONUS,TOTAL_CURR,AGE,EXECRANKANN,TDC1,ALLOTHTOT,ALLOTHPD,...,DIRSTK,DIRSTKAD,PDIRPENSN,OIBD,PRETAX,NIBEX,MKTVAL,directors_total_comp,director_comp_x,director_comp_y
count,59752.000000,59752.000000,59752.000000,59752.000000,59752.000000,55482.000000,59640.000000,59276.000000,59752.000000,59752.000000,...,55087.000000,55087.000000,59752.000000,56161.000000,57219.000000,57219.000000,57252.000000,26514.000000,26514.000000,59752.000000
mean,0.988754,7942.941893,798.104714,1200.643885,1998.748599,56.477921,1.273709,7091.134426,238.012476,101.817006,...,0.500459,0.169981,0.177383,2238.828414,1248.823505,811.712059,18360.204858,42.571266,42.571266,3312.646793
std,0.105452,8119.664945,431.752567,1882.064622,2087.461908,7.053710,0.830027,14439.864295,1837.301301,1714.795104,...,1.085606,0.988589,0.381996,4737.499452,3011.722682,2027.280317,40636.060737,24.983141,24.983141,3424.132830
min,0.000000,1.000000,0.000000,0.000000,0.000000,29.000000,1.000000,0.001000,0.000000,0.000000,...,0.000000,0.000000,0.000000,-1728.000000,-21176.000000,-21176.000000,6.268000,2.000000,2.000000,0.000000
25%,1.000000,1553.000000,531.250000,289.916250,900.000000,52.000000,1.000000,1884.862000,8.442750,0.000000,...,0.000000,0.000000,0.000000,273.089000,127.575000,80.730000,2256.483000,26.400000,26.400000,134.676000
50%,1.000000,4018.000000,763.731500,686.169500,1472.120500,57.000000,1.000000,3882.614000,39.377000,0.000000,...,0.000000,0.000000,0.000000,806.000000,430.892000,278.311000,5855.900000,38.100000,38.100000,2363.900000
75%,1.000000,13856.750000,995.000000,1415.650000,2376.225000,61.000000,1.000000,8043.537000,121.894250,0.000000,...,0.600000,0.000000,0.000000,2056.000000,1138.466000,731.523000,14858.945000,53.500000,53.500000,5160.984000
max,1.000000,36182.000000,5806.651000,43511.534000,43511.535000,83.000000,8.000000,600347.351000,95195.500000,95107.134000,...,75.000000,30.047000,1.000000,59255.000000,60231.000000,36130.000000,507216.647000,340.000000,340.000000,20388.662000


In [ ]:
# 2010- 2016, 2016- 2019

In [24]:
merged_df['total_director_comp'].describe()

KeyError: 'total_director_comp'

In [26]:
merged_df = merged_df[merged_df['CEOANN'].notnull()]


In [27]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59752 entries, 0 to 59751
Data columns (total 53 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   EXECDIR               59752 non-null  int64  
 1   CO_PER_ROL            59752 non-null  int64  
 2   CONAME                59752 non-null  object 
 3   CEOANN                59752 non-null  object 
 4   SALARY                59752 non-null  float64
 5   BONUS                 59752 non-null  float64
 6   TOTAL_CURR            59752 non-null  float64
 7   AGE                   55482 non-null  float64
 8   EXECRANKANN           59640 non-null  float64
 9   TDC1                  59276 non-null  float64
 10  ALLOTHTOT             59752 non-null  float64
 11  ALLOTHPD              59752 non-null  float64
 12  SAL_PCT               56700 non-null  float64
 13  TOTAL_CURR_PCT        56686 non-null  float64
 14  TDC1_PCT              55566 non-null  float64
 15  COMMENT            